# Sense HAT for PYNQ:Temperature and Pressure Sensor

This notebook illustrates how to read the temperature and pressure sensor data use the [Sense HAT](https://www.raspberrypi.org/products/sense-hat/) .


This example notebook includes the following steps.
1. import python libraries
2. select RPi switch and using Microblaze library
3. configure the I2C device
4. read single temperature and pressure
5. read and plot temperature once every 200ms for 5s

![PYNQ with Sense HAT](data/PYNQ_with_Sense_HAT.jpg)


### 1. Sense HAT Introduction

The Sense HAT, which is a fundamental part of the [Astro Pi](https://astro-pi.org/) mission, allows your board to sense the world around it. It has a 8×8 RGB LED matrix, a five-button joystick and includes the following sensors:

* Gyroscope
* Accelerometer
* Magnetometer
* Temperature
* Barometric pressure
* Humidity

![Sense HAT add-on board](data/Sense_HAT.jpg)


### 2. Prepare the overlay
Download the overlay first, then select the shared pin to be connected to
RPi header (by default, the pins will be connected to PMODA instead).

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
import matplotlib.pyplot as plt
from imp import reload
from time import sleep
from sensehat import *

In [ ]:
base = BaseOverlay('base.bit')
lib = MicroblazeLibrary(base.RPI, ['i2c', 'gpio', 'xio_switch','circular_buffer'])

### 3. Configure the I2C device and GPIO device
Initialize the I2C device and set the I2C pin of RPi header. Since the PYNQ-ZU board does not have pull-up on the Reset_N pin of the HAT (GPIO25), set that to 1. 

In [ ]:
i2c = lib.i2c_open_device(1)
lib.set_pin(2, lib.SDA1)
lib.set_pin(3, lib.SCL1)
gpio = lib.gpio_open(25)
gpio.write(1)

### 4. Read single temperature and pressure
The MEMS pressure sensor of Sense HAT is LPS25H, 

In [ ]:
lps25h_sensor = lps25h.LPS25H_I2C(i2c)
press = lps25h_sensor.pressure
print('Pressure (hPa): ({0:0.3f})'.format(press))
tmp = lps25h_sensor.temperature
print('Temperature (℃"): ({0:0.3f})'.format(tmp))

### 5. Start logging once every 200ms for 5 seconds
Executing the next cell will start logging the temperature sensor values every 200ms, and will run for 5s. 

In [ ]:
cnt = 0
tmp_array = []
while True:
    tmp = lps25h_sensor.temperature
    tmp_array.append(tmp)
    cnt = cnt + 1
    sleep(0.2)
    if cnt > 25:
        break
plt.plot(range(len(tmp_array)), tmp_array, 'ro')
plt.title("Sense Hat Temperature Plot")
min_tmp_array = min(tmp_array)
max_tmp_array = max(tmp_array)
plt.axis([0, len(tmp_array), min_tmp_array, max_tmp_array])
plt.show()

### 6.Clean up
Close the I2C device and switch back the connection on the shared pin to PMODA header.

In [ ]:
i2c.close()

Copyright (C) 2020 Xilinx, Inc